# 2クラス分類分析

このNotebookでは2クラス分類モデルを作成し、Recallを算出します。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")

## 1. データの読み込み

In [ ]:
# CSVデータを読み込み
df = pd.read_csv('binary_classification_data.csv')

print(f"データ形状: {df.shape}")
print(f"\nクラス分布:")
print(df['target'].value_counts())

# データの最初の数行を表示
print("\nデータの最初の5行:")
df.head()

In [ ]:
# データの基本統計情報
df.describe()

## 2. 学習データと評価データに分割

In [ ]:
# 特徴量とターゲットを分離
X = df.drop('target', axis=1)
y = df['target']

# 学習データと評価データに分割（8:2の比率）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"学習データ形状: {X_train.shape}")
print(f"評価データ形状: {X_test.shape}")
print(f"\n学習データのクラス分布:")
print(y_train.value_counts())
print(f"\n評価データのクラス分布:")
print(y_test.value_counts())

## 3. モデルの学習

In [ ]:
# Random Forestモデル
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print("Random Forestモデルの学習完了")

In [ ]:
# データの標準化（Logistic RegressionとSVM用）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regressionモデル
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_scaled, y_train)

# SVMモデル
svm_model = SVC(random_state=42, probability=True)
svm_model.fit(X_train_scaled, y_train)

print("Logistic RegressionとSVMモデルの学習完了")

## 4. モデルの評価とRecallの算出

In [ ]:
# Random Forestの評価
print("=== Random Forest の評価結果 ===")
rf_pred = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)  # 重要: Recallの算出
rf_f1 = f1_score(y_test, rf_pred)

print(f"Accuracy: {rf_accuracy:.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")
print(f"F1-Score: {rf_f1:.4f}")

print("\n詳細な分類レポート:")
print(classification_report(y_test, rf_pred))

In [ ]:
# Random Forestの混同行列
plt.figure(figsize=(6, 4))
cm_rf = confusion_matrix(y_test, rf_pred)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Random Forest - 混同行列')
plt.ylabel('実際のクラス')
plt.xlabel('予測クラス')
plt.show()

In [ ]:
# Logistic Regressionの評価
print("=== Logistic Regression の評価結果 ===")
lr_pred = lr_model.predict(X_test_scaled)

lr_accuracy = accuracy_score(y_test, lr_pred)
lr_precision = precision_score(y_test, lr_pred)
lr_recall = recall_score(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)

print(f"Accuracy: {lr_accuracy:.4f}")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall: {lr_recall:.4f}")
print(f"F1-Score: {lr_f1:.4f}")

print("\n詳細な分類レポート:")
print(classification_report(y_test, lr_pred))

In [ ]:
# Logistic Regressionの混同行列
plt.figure(figsize=(6, 4))
cm_lr = confusion_matrix(y_test, lr_pred)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues')
plt.title('Logistic Regression - 混同行列')
plt.ylabel('実際のクラス')
plt.xlabel('予測クラス')
plt.show()

In [ ]:
# SVMの評価
print("=== SVM の評価結果 ===")
svm_pred = svm_model.predict(X_test_scaled)

svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred)
svm_recall = recall_score(y_test, svm_pred)
svm_f1 = f1_score(y_test, svm_pred)

print(f"Accuracy: {svm_accuracy:.4f}")
print(f"Precision: {svm_precision:.4f}")
print(f"Recall: {svm_recall:.4f}")
print(f"F1-Score: {svm_f1:.4f}")

print("\n詳細な分類レポート:")
print(classification_report(y_test, svm_pred))

In [ ]:
# SVMの混同行列
plt.figure(figsize=(6, 4))
cm_svm = confusion_matrix(y_test, svm_pred)
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Blues')
plt.title('SVM - 混同行列')
plt.ylabel('実際のクラス')
plt.xlabel('予測クラス')
plt.show()

## 5. モデルの比較

In [ ]:
# 結果をまとめて比較
results_df = pd.DataFrame({
    'Random Forest': [rf_accuracy, rf_precision, rf_recall, rf_f1],
    'Logistic Regression': [lr_accuracy, lr_precision, lr_recall, lr_f1],
    'SVM': [svm_accuracy, svm_precision, svm_recall, svm_f1]
}, index=['Accuracy', 'Precision', 'Recall', 'F1-Score'])

print("=== モデル比較 ===")
print(results_df.round(4))

# Recallでソートして最良モデルを特定
best_model_recall = results_df.loc['Recall'].idxmax()
best_recall_value = results_df.loc['Recall'].max()

print(f"\n最高Recall: {best_model_recall} (Recall: {best_recall_value:.4f})")

In [ ]:
# 視覚的な比較
plt.figure(figsize=(12, 8))
results_df.T.plot(kind='bar', figsize=(12, 6))
plt.title('モデル性能比較')
plt.xlabel('モデル')
plt.ylabel('スコア')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

## 6. Recallに特化した分析

In [ ]:
# Recallの詳細分析
print("=== Recall詳細分析 ===")
print(f"Random Forest Recall: {rf_recall:.4f}")
print(f"Logistic Regression Recall: {lr_recall:.4f}")
print(f"SVM Recall: {svm_recall:.4f}")

# Recallの意味を解説
print("\nRecallとは:")
print("実際にPositive（クラス1）であるもののうち、正しくPositiveと予測できた割合")
print("Recall = TP / (TP + FN)")
print("TP: True Positive, FN: False Negative")

# 各モデルのTP, FNを計算
models_data = [
    ('Random Forest', rf_pred),
    ('Logistic Regression', lr_pred),
    ('SVM', svm_pred)
]

print("\n各モデルのTP, FN, Recall詳細:")
for model_name, pred in models_data:
    cm = confusion_matrix(y_test, pred)
    tn, fp, fn, tp = cm.ravel()
    recall = tp / (tp + fn)
    print(f"{model_name}: TP={tp}, FN={fn}, Recall={recall:.4f}")

## まとめ

この分析では以下を実施しました：

1. **仮想的なCSVデータの作成**: 2クラス分類用の1000サンプル、10特徴量のデータセット
2. **学習データと評価データの分割**: 8:2の比率で分割
3. **複数のモデルの学習**:
   - Random Forest
   - Logistic Regression
   - SVM
4. **Recallの算出と比較**: 各モデルのRecall性能を評価

**重要なポイント**:
- Recallは「実際のPositiveクラスをどれだけ正しく検出できたか」を示す指標
- 医療診断や不正検知など、見逃しを避けたい場面で重要
- モデル選択時はRecallだけでなく、Precision、F1-Scoreも考慮して総合的に判断